In [1]:
import pandas as pd
import time
import requests
import json
from tqdm import tqdm

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
options.add_argument("headless")
s = Service(ChromeDriverManager().install())
chrome = webdriver.Chrome(options = options,service = s)



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [/home/cosbi/.wdm/drivers/chromedriver/linux64/106.0.5249.61/chromedriver] found in cache


In [14]:
chrome.get("https://tw.stock.yahoo.com/h/kimosel.php?tse=2&cat=%E6%AB%83%E5%8D%8A%E5%B0%8E&form=menu&form_id=stock_id&form_name=stock_name&domain=0")

In [21]:
ticker = {}
td_text = chrome.find_elements(by = By.TAG_NAME, value = "td")[3].text
tiker_row = td_text.split("\n")
tiker_row[0] = tiker_row[0].replace("上市 ", "")
tiker_row[0] = tiker_row[0].replace(" 上櫃", "")
tiker_row[len(tiker_row) - 1] = tiker_row[len(tiker_row) - 1][:-8]

In [23]:
for row in tiker_row:
    cols = row.split(" ")

    for col in cols: ticker[col] = []

In [25]:
for key in tqdm(ticker.keys()):
    chrome.get("https://tw.stock.yahoo.com/h/kimosel.php?tse=2&cat=" + key + "&form=menu&form_id=stock_id&form_name=stock_name&domain=0")
    tables_tag = chrome.find_elements(by = By.TAG_NAME, value = "table")[6]
    a_tags = tables_tag.find_elements(by = By.TAG_NAME, value = "a")

    for a_tag in a_tags: ticker[key].append(a_tag.text)

    time.sleep(0.5)


100%|██████████| 32/32 [03:44<00:00,  7.01s/it]


In [26]:
a = []
b = []
c = []

for key in ticker.keys():
    for ele in ticker[key]:
        a.append(ele)
        b.append(ele.split(" ")[0])

    c.extend([key for i in range(len(ticker[key]))])

In [27]:
df = pd.DataFrame({ "stock_name" : a, "stock_num" : b, "class" : c })
df

,stock_name,stock_num,class
0,EMP88 就業88,EMP88,櫃指數類
1,GTCI 薪酬指數,GTCI,櫃指數類
2,GTHD 高殖利,GTHD,櫃指數類
3,GTSM50 富櫃50,GTSM50,櫃指數類
4,IR0118 富櫃200報酬指,IR0118,櫃指數類
...,...,...,...
7909,72006P 威剛富邦25售01,72006P,櫃認售
7910,72007P 元太群益25售13,72007P,櫃認售
7911,72008P 鈺創群益25售03,72008P,櫃認售
7912,72009P 優群群益25售02,72009P,櫃認售


In [9]:
df.to_csv("ticker_list.csv", index = False)